# Importando bibliotecas

In [1]:
#!pip install i18n
#!pip install dotenv

In [1]:
#!pip install pandas
#!pip install investpy
#!pip install selenium
#!pip install webdriver-manager

In [2]:
import pandas as pd
import numpy as np

from datetime import datetime

#!pip install investpy
#import investpy as inv
from time import sleep
import time

In [1]:
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome import service
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.ui import Select
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

In [3]:
from selenium import webdriver

# Funcoes criadas

In [4]:
def converter_valor(valor):
    if valor.endswith('B'):
        return float(valor.replace('B', '').replace(' ', '')) * 1_000
    elif valor.endswith('M'):
        return float(valor.replace('M', '').replace(' ', '')) * 1
    elif valor.endswith('K'):
        return float(valor.replace('K', '').replace(' ', '')) * 0.001
    elif valor == 'NA':
        return float('nan')

# Pegando os dados

In [5]:
codigos_ibovespa = [
    'ABEV3', 'AZUL4', 'B3SA3', 'BBAS3', 'BBDC3', 'BBDC4', 'BBSE3', 'BEEF3', 'BPAC11', 'BRAP4',
    #'VVBR3', 'BTOW3','ENBR3',GNDI3,IGTA3
    'BRFS3', 'BRKM5',  'CCRO3', 'CIEL3', 'CMIG4', 'COGN3', 'CPFE3',
    'CRFB3', 'CSAN3', 'CSNA3', 'CVCB3', 'CYRE3', 'ECOR3', 'EGIE3', 'ELET3', 'ELET6', 'EMBR3',
     'ENGI11', 'EQTL3', 'EZTC3', 'FLRY3', 'GGBR4',  'GOAU4', 'GOLL4', 'HAPV3',
     'HYPE3', 'IRBR3', 'ITSA4', 'ITUB4', 'JBSS3', 'KLBN11', 'LREN3',
    'MGLU3', 'MRFG3', 'MRVE3', 'MULT3', 'NTCO3', 'PCAR3', 'PETR3', 'PETR4', 'PRIO3', 'QUAL3',
    'RADL3', 'RAIL3', 'RENT3', 'SANB11', 'SBSP3','SUZB3', 'TAEE11', 'TIMS3', 'TOTS3',
    'UGPA3', 'USIM5', 'VALE3', 'VIVT3', 'WEGE3', 'WIZC3','YDUQ3','SOMA3','BPAC11','MDIA3'
]

# Fazer itercao dao comeco

In [6]:
refazer_comeco = True
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service

In [7]:
codigos_ibovespa = [
    'ABEV3', 'AZUL4', 'B3SA3','BBDC4']
if refazer_comeco:    
    chrome_driver_path = "/usr/bin/chromedriver"

    service = Service(executable_path=chrome_driver_path)

    options = Options()
    options.add_argument("--headless")  # Executar em modo headless
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument("--disable-dev-shm-usage")  # Reduz o uso de memória
    options.add_argument("--disable-gpu")  # Desativar o uso de GPU
    options.add_argument("--window-size=1920,1080")  # Definir o tamanho da janela
    #setor_financeiro = ['BBAS3','BBDC3']
    setor_financeiro = {'BBAS3', 'BBDC3','BBDC4','BBSE3','ITUB4','BPAC11'}
    lista_dataframes = []
    navegador = webdriver.Chrome(service=service, options=options)
    iteracao=0
    for acao in codigos_ibovespa:
        if acao in setor_financeiro:
            print(f'Execucao da {acao} do setor financeiro.')
            start_time = time.time() 
            navegador.get(f'https://www.investsite.com.br/principais_indicadores.php?cod_negociacao={acao}')
            
            # Extrair o texto de todas as datas
            elementos_data = navegador.find_elements(By.XPATH, '//*[@id="tabela_resumo_empresa_dre_3meses"]/thead/tr/th[2]/select/option')
            datas = [elemento.text for elemento in elementos_data]

            # Encontrar a tabela com o id
            tabela = navegador.find_element(By.ID, 'tabela_resumo_empresa_dre_3meses')
            receita_liquida = []
            resultado_bruto  = []
            lista_ebit = []
            depreciacao_amortizacao = []
            lista_ebitda = []
            lucro_liquido = []
            lucro_acao = []
            for i in datas:
                elemento_select = navegador.find_element(By.XPATH, '/html/body/div[1]/div[4]/div[2]/div[1]/div[3]/table[2]/thead/tr/th[2]/select')
                # Crie um objeto Select
                select = Select(elemento_select)
                # Altere o período selecionando por texto visível
                select.select_by_visible_text(i)
                servico = webdriver.chrome.service.Service(ChromeDriverManager().install())

                # Pegar as linhas da tabela
                linhas = tabela.find_elements(By.TAG_NAME, 'tr')

                # Iterar pelas linhas e capturar os dados
                lista_dre_ultimo_mes = []
                for linha in linhas:
                    # Pegar as células de cada linha
                    celulas = linha.find_elements(By.TAG_NAME, 'td')
                    for celula in celulas:
                        lista_dre_ultimo_mes.append(celula.text)


                receita_liquida.append(lista_dre_ultimo_mes[1].replace('R$','').replace(',','.').strip())
                resultado_bruto.append(lista_dre_ultimo_mes[3].replace('R$','').replace(',','.').strip())
                lucro_liquido.append(lista_dre_ultimo_mes[7].replace('R$','').replace(',','.').strip())
                lista_ebit.append(np.nan)
                depreciacao_amortizacao.append(np.nan)
                lista_ebitda.append(np.nan)
                lucro_acao.append(np.nan)

        else:
            print(f'Execucao da {acao}')
            start_time = time.time() 
            navegador.get(f'https://www.investsite.com.br/principais_indicadores.php?cod_negociacao={acao}')
            #tabela = navegador.find_element(By.XPATH, '//*[@id="tabela_resumo_empresa_dre_3meses"]')

            # Extrair o texto de todas as datas
            elementos_data = navegador.find_elements(By.XPATH, '//*[@id="tabela_resumo_empresa_dre_3meses"]/thead/tr/th[2]/select/option')
            datas = [elemento.text for elemento in elementos_data]

            # Encontrar a tabela com o id
            tabela = navegador.find_element(By.ID, 'tabela_resumo_empresa_dre_3meses')
            receita_liquida = []
            resultado_bruto  = []
            lista_ebit = []
            depreciacao_amortizacao = []
            lista_ebitda = []
            lucro_liquido = []
            lucro_acao = []
            for i in datas:
                elemento_select = navegador.find_element(By.XPATH, '/html/body/div[1]/div[4]/div[2]/div[1]/div[3]/table[2]/thead/tr/th[2]/select')#/html/body/div[1]/div[4]/div[2]/div[1]/div[3]/table[2]/thead/tr/th[2]/select
                # Crie um objeto Select
                select = Select(elemento_select)
                # Altere o período selecionando por texto visível
                select.select_by_visible_text(i)
                servico = webdriver.chrome.service.Service(ChromeDriverManager().install())

                # Pegar as linhas da tabela
                linhas = tabela.find_elements(By.TAG_NAME, 'tr')

                # Iterar pelas linhas e capturar os dados
                lista_dre_ultimo_mes = []
                for linha in linhas:
                    # Pegar as células de cada linha
                    celulas = linha.find_elements(By.TAG_NAME, 'td')
                    for celula in celulas:
                        lista_dre_ultimo_mes.append(celula.text)
                receita_liquida.append(lista_dre_ultimo_mes[1].replace('R$','').replace(',','.').strip())
                resultado_bruto.append(lista_dre_ultimo_mes[3].replace('R$','').replace(',','.').strip())
                lista_ebit.append(lista_dre_ultimo_mes[5].replace('R$','').replace(',','.').strip())
                depreciacao_amortizacao.append(lista_dre_ultimo_mes[7].replace('R$','').replace(',','.').strip())
                lista_ebitda.append(lista_dre_ultimo_mes[9].replace('R$','').replace(',','.').strip())
                lucro_liquido.append(lista_dre_ultimo_mes[11].replace('R$','').replace(',','.').strip())
                lucro_acao.append(lista_dre_ultimo_mes[13].replace('R$','').replace(',','.').strip())

        listas_dados = {
            'data':datas,
            'receita_liquida': receita_liquida,
            'resultado_bruto': resultado_bruto,
            'ebit': lista_ebit,
            'depreciacao_amortizacao': depreciacao_amortizacao,
            'ebitida': lista_ebitda,
            'lucro_liquido': lucro_liquido,
            'lucro_por_acao': lucro_acao
        }
        df_dre_ultimo_mes = pd.DataFrame(listas_dados)
        df_dre_ultimo_mes['data'] = pd.to_datetime(df_dre_ultimo_mes['data'], format='%d/%m/%Y')
        df_dre_ultimo_mes['data'] = df_dre_ultimo_mes['data'].dt.strftime('%Y-%m-%d')

        if acao in setor_financeiro:
            colunas_selecionadas = df_dre_ultimo_mes.columns[(df_dre_ultimo_mes.columns != 'data') & 
                                    (df_dre_ultimo_mes.columns != 'lucro_por_acao') &
                                    (df_dre_ultimo_mes.columns != 'ebit') &   
                                    (df_dre_ultimo_mes.columns != 'depreciacao_amortizacao') & 
                                    (df_dre_ultimo_mes.columns != 'lucro_por_acao') &
                                    (df_dre_ultimo_mes.columns != 'ebitida') &  
                                    (df_dre_ultimo_mes.columns != 'lucro_por_acao')]
            for col in colunas_selecionadas:
                df_dre_ultimo_mes[col] = df_dre_ultimo_mes[col].apply(converter_valor)   
        else:
            for col in df_dre_ultimo_mes.columns[(df_dre_ultimo_mes.columns != 'data') & (df_dre_ultimo_mes.columns != 'lucro_por_acao')]:
                df_dre_ultimo_mes[col] = df_dre_ultimo_mes[col].apply(converter_valor)

        df_dre_ultimo_mes['tic'] = acao

        lista_dataframes.append(df_dre_ultimo_mes)

        iteracao = iteracao+1

        if iteracao % 15 == 0:
            pd.concat(lista_dataframes).to_csv('dre_trimestral.csv')

        end_time = time.time()

        # Calcula o tempo decorrido em segundos
        execution_time_seconds = end_time - start_time

        # Calcula o tempo decorrido em minutos
        execution_time_minutes = execution_time_seconds / 60

        # Imprime o tempo de execução em minutos
        print(f"Tempo de execução da ação {acao}: {execution_time_minutes:.2f} minutos")

Execucao da ABEV3
Tempo de execução da ação ABEV3: 1.06 minutos
Execucao da AZUL4
Tempo de execução da ação AZUL4: 0.68 minutos
Execucao da B3SA3
Tempo de execução da ação B3SA3: 1.49 minutos
Execucao da BBDC4 do setor financeiro.
Tempo de execução da ação BBDC4: 2.00 minutos


In [11]:
df_resumo_balanco = pd.concat(lista_dataframes)

In [12]:
df_resumo_balanco.columns[(df_resumo_balanco.columns != 'data') & (df_resumo_balanco.columns != 'lucro_por_acao') & (df_resumo_balanco.columns != 'tic')]

Index(['receita_liquida', 'resultado_bruto', 'ebit', 'depreciacao_amortizacao',
       'ebitida', 'lucro_liquido'],
      dtype='object')

# Fazer da onde parou

In [9]:
data_frame_feito = pd.read_csv('dre_trimestral_2.csv').iloc[:,1:]

In [16]:
data_frame_feito.tic.unique()

array(['ABEV3', 'AZUL4', 'B3SA3', 'BBAS3', 'BBDC3', 'BBDC4', 'BBSE3',
       'BEEF3', 'BPAC11', 'BRAP4', 'BRFS3', 'BRKM5', 'CCRO3', 'CIEL3',
       'CMIG4', 'COGN3', 'CPFE3', 'CRFB3', 'CSAN3', 'CSNA3', 'CVCB3',
       'CYRE3', 'ECOR3', 'EGIE3', 'ELET3', 'ELET6', 'EMBR3', 'ENGI11',
       'EQTL3', 'EZTC3', 'FLRY3', 'GGBR4', 'GOAU4', 'GOLL4', 'HAPV3',
       'HYPE3', 'IRBR3', 'ITSA4', 'ITUB4', 'JBSS3', 'KLBN11', 'LREN3',
       'MGLU3', 'MRFG3', 'MRVE3', 'MULT3', 'NTCO3', 'PCAR3', 'PETR3',
       'PETR4', 'PRIO3', 'QUAL3', 'RADL3', 'RAIL3', 'RENT3', 'SANB11',
       'SBSP3', 'SUZB3', 'TAEE11', 'TIMS3', 'TOTS3', 'UGPA3', 'USIM5'],
      dtype=object)

In [17]:
lista_dataframes = []
#setor_financeiro = ['BBAS3','BBDC3']
setor_financeiro = {'BBAS3', 'BBDC3','BBDC4','BBSE3','ITUB4','BPAC11','ITUB4','SANB11','IRBR3'}
iteracao=0
for acao in codigos_ibovespa:
    if acao in data_frame_feito['tic'].unique():
        lista_dataframes.append(data_frame_feito[data_frame_feito.tic == acao])
        print(f'Essa {acao} ja foi')
    else:
        servico = webdriver.chrome.service.Service(ChromeDriverManager().install())
        # Iniciar o navegador com o serviço
        navegador = webdriver.Chrome(service=servico)
      
        
        if acao in setor_financeiro:
            print(f'Execucao da {acao} do setor financeiro.')
            start_time = time.time() 
            navegador.get(f'https://www.investsite.com.br/principais_indicadores.php?cod_negociacao={acao}')
            #tabela = navegador.find_element(By.XPATH, '//*[@id="tabela_resumo_empresa_dre_3meses"]')

            # Extrair o texto de todas as datas
            elementos_data = navegador.find_elements(By.XPATH, '//*[@id="tabela_resumo_empresa_dre_3meses"]/thead/tr/th[2]/select/option')
            datas = [elemento.text for elemento in elementos_data]

            # Encontrar a tabela com o id
            tabela = navegador.find_element(By.ID, 'tabela_resumo_empresa_dre_3meses')
            receita_liquida = []
            resultado_bruto  = []
            lista_ebit = []
            depreciacao_amortizacao = []
            lista_ebitda = []
            lucro_liquido = []
            lucro_acao = []
            for i in datas:
                elemento_select = navegador.find_element(By.XPATH, '/html/body/div[1]/div[4]/div[2]/div[1]/div[3]/table[2]/thead/tr/th[2]/select')
                # Crie um objeto Select
                select = Select(elemento_select)
                # Altere o período selecionando por texto visível
                select.select_by_visible_text(i)
                servico = webdriver.chrome.service.Service(ChromeDriverManager().install())

                # Pegar as linhas da tabela
                linhas = tabela.find_elements(By.TAG_NAME, 'tr')

                # Iterar pelas linhas e capturar os dados
                lista_dre_ultimo_mes = []
                for linha in linhas:
                    # Pegar as células de cada linha
                    celulas = linha.find_elements(By.TAG_NAME, 'td')
                    for celula in celulas:
                        lista_dre_ultimo_mes.append(celula.text)


                receita_liquida.append(lista_dre_ultimo_mes[1].replace('R$','').replace(',','.').strip())
                resultado_bruto.append(lista_dre_ultimo_mes[3].replace('R$','').replace(',','.').strip())
                lucro_liquido.append(lista_dre_ultimo_mes[7].replace('R$','').replace(',','.').strip())
                lista_ebit.append(np.nan)
                depreciacao_amortizacao.append(np.nan)
                lista_ebitda.append(np.nan)
                lucro_acao.append(np.nan)
        elif acao in ['IRBR3']:
            
            print(f'Execucao da {acao}')
            start_time = time.time() 
            navegador.get(f'https://www.investsite.com.br/principais_indicadores.php?cod_negociacao={acao}')
            #tabela = navegador.find_element(By.XPATH, '//*[@id="tabela_resumo_empresa_dre_3meses"]')

            # Extrair o texto de todas as datas
            elementos_data = navegador.find_elements(By.XPATH, '//*[@id="tabela_resumo_empresa_dre_3meses"]/thead/tr/th[2]/select/option')
            datas = [elemento.text for elemento in elementos_data]

            # Encontrar a tabela com o id
            tabela = navegador.find_element(By.ID, 'tabela_resumo_empresa_dre_3meses')
            receita_liquida = []
            resultado_bruto  = []
            lista_ebit = []
            depreciacao_amortizacao = []
            lista_ebitda = []
            lucro_liquido = []
            lucro_acao = []
            for i in datas:
                elemento_select = navegador.find_element(By.XPATH, '/html/body/div[1]/div[4]/div[2]/div[1]/div[3]/table[2]/thead/tr/th[2]/select')#/html/body/div[1]/div[4]/div[2]/div[1]/div[3]/table[2]/thead/tr/th[2]/select
                # Crie um objeto Select
                select = Select(elemento_select)
                # Altere o período selecionando por texto visível
                select.select_by_visible_text(i)
                servico = webdriver.chrome.service.Service(ChromeDriverManager().install())

                # Pegar as linhas da tabela
                linhas = tabela.find_elements(By.TAG_NAME, 'tr')

                # Iterar pelas linhas e capturar os dados
                lista_dre_ultimo_mes = []
                for linha in linhas:
                    # Pegar as células de cada linha
                    celulas = linha.find_elements(By.TAG_NAME, 'td')
                    for celula in celulas:
                        lista_dre_ultimo_mes.append(celula.text)
                receita_liquida.append(lista_dre_ultimo_mes[1].replace('R$','').replace(',','.').strip())
                resultado_bruto.append(lista_dre_ultimo_mes[3].replace('R$','').replace(',','.').strip())
                lista_ebit.append(lista_dre_ultimo_mes[5].replace('R$','').replace(',','.').strip())
                depreciacao_amortizacao.append(np.nan)
                lista_ebitda.append(np.nan)
                lucro_liquido.append(lista_dre_ultimo_mes[7].replace('R$','').replace(',','.').strip())
                lucro_acao.append(np.nan)

        else:
            print(f'Execucao da {acao}')
            start_time = time.time() 
            navegador.get(f'https://www.investsite.com.br/principais_indicadores.php?cod_negociacao={acao}')
            #tabela = navegador.find_element(By.XPATH, '//*[@id="tabela_resumo_empresa_dre_3meses"]')

            # Extrair o texto de todas as datas
            elementos_data = navegador.find_elements(By.XPATH, '//*[@id="tabela_resumo_empresa_dre_3meses"]/thead/tr/th[2]/select/option')
            datas = [elemento.text for elemento in elementos_data]

            # Encontrar a tabela com o id
            tabela = navegador.find_element(By.ID, 'tabela_resumo_empresa_dre_3meses')
            receita_liquida = []
            resultado_bruto  = []
            lista_ebit = []
            depreciacao_amortizacao = []
            lista_ebitda = []
            lucro_liquido = []
            lucro_acao = []
            for i in datas:
                elemento_select = navegador.find_element(By.XPATH, '/html/body/div[1]/div[4]/div[2]/div[1]/div[3]/table[2]/thead/tr/th[2]/select')#/html/body/div[1]/div[4]/div[2]/div[1]/div[3]/table[2]/thead/tr/th[2]/select
                # Crie um objeto Select
                select = Select(elemento_select)
                # Altere o período selecionando por texto visível
                select.select_by_visible_text(i)
                servico = webdriver.chrome.service.Service(ChromeDriverManager().install())

                # Pegar as linhas da tabela
                linhas = tabela.find_elements(By.TAG_NAME, 'tr')

                # Iterar pelas linhas e capturar os dados
                lista_dre_ultimo_mes = []
                for linha in linhas:
                    # Pegar as células de cada linha
                    celulas = linha.find_elements(By.TAG_NAME, 'td')
                    for celula in celulas:
                        lista_dre_ultimo_mes.append(celula.text)
                receita_liquida.append(lista_dre_ultimo_mes[1].replace('R$','').replace(',','.').strip())
                resultado_bruto.append(lista_dre_ultimo_mes[3].replace('R$','').replace(',','.').strip())
                lista_ebit.append(lista_dre_ultimo_mes[5].replace('R$','').replace(',','.').strip())
                depreciacao_amortizacao.append(lista_dre_ultimo_mes[7].replace('R$','').replace(',','.').strip())
                lista_ebitda.append(lista_dre_ultimo_mes[9].replace('R$','').replace(',','.').strip())
                lucro_liquido.append(lista_dre_ultimo_mes[11].replace('R$','').replace(',','.').strip())
                lucro_acao.append(lista_dre_ultimo_mes[13].replace('R$','').replace(',','.').strip())

        listas_dados = {
                'data':datas,
                'receita_liquida': receita_liquida,
                'resultado_bruto': resultado_bruto,
                'ebit': lista_ebit,
                'depreciacao_amortizacao': depreciacao_amortizacao,
                'ebitida': lista_ebitda,
                'lucro_liquido': lucro_liquido,
                'lucro_por_acao': lucro_acao
            }
        df_dre_ultimo_mes = pd.DataFrame(listas_dados)
        df_dre_ultimo_mes['data'] = pd.to_datetime(df_dre_ultimo_mes['data'], format='%d/%m/%Y')
        df_dre_ultimo_mes['data'] = df_dre_ultimo_mes['data'].dt.strftime('%Y-%m-%d')

        if acao in setor_financeiro:
            colunas_selecionadas = df_dre_ultimo_mes.columns[(df_dre_ultimo_mes.columns != 'data') & 
                                        (df_dre_ultimo_mes.columns != 'lucro_por_acao') &
                                        (df_dre_ultimo_mes.columns != 'ebit') &   
                                        (df_dre_ultimo_mes.columns != 'depreciacao_amortizacao') & 
                                        (df_dre_ultimo_mes.columns != 'lucro_por_acao') &
                                        (df_dre_ultimo_mes.columns != 'ebitida') &  
                                        (df_dre_ultimo_mes.columns != 'lucro_por_acao')]
            for col in colunas_selecionadas:
                df_dre_ultimo_mes[col] = df_dre_ultimo_mes[col].apply(converter_valor)  

        elif acao in ['IRBR3']:
            for col in df_dre_ultimo_mes.columns[(df_dre_ultimo_mes.columns == 'receita_liquida') | (df_dre_ultimo_mes.columns == 'resultado_bruto')|
                                                (df_dre_ultimo_mes.columns == 'ebit') | (df_dre_ultimo_mes.columns == 'lucro_liquido')]:
                df_dre_ultimo_mes[col] = df_dre_ultimo_mes[col].apply(converter_valor)
            
        else:
            for col in df_dre_ultimo_mes.columns[(df_dre_ultimo_mes.columns != 'data') & (df_dre_ultimo_mes.columns != 'lucro_por_acao')]:
                df_dre_ultimo_mes[col] = df_dre_ultimo_mes[col].apply(converter_valor)

        df_dre_ultimo_mes['tic'] = acao

        lista_dataframes.append(df_dre_ultimo_mes)

        iteracao = iteracao+1

        if iteracao % 5 == 0:
            pd.concat(lista_dataframes).to_csv('dre_trimestral.csv')

        end_time = time.time()

        # Calcula o tempo decorrido em segundos
        execution_time_seconds = end_time - start_time

        # Calcula o tempo decorrido em minutos
        execution_time_minutes = execution_time_seconds / 60

        # Imprime o tempo de execução em minutos
        print(f"Tempo de execução da ação {acao}: {execution_time_minutes:.2f} minutos")   

Essa ABEV3 ja foi
Essa AZUL4 ja foi
Essa B3SA3 ja foi
Essa BBAS3 ja foi
Essa BBDC3 ja foi
Essa BBDC4 ja foi
Essa BBSE3 ja foi
Essa BEEF3 ja foi
Essa BPAC11 ja foi
Essa BRAP4 ja foi
Essa BRFS3 ja foi
Essa BRKM5 ja foi
Essa CCRO3 ja foi
Essa CIEL3 ja foi
Essa CMIG4 ja foi
Essa COGN3 ja foi
Essa CPFE3 ja foi
Essa CRFB3 ja foi
Essa CSAN3 ja foi
Essa CSNA3 ja foi
Essa CVCB3 ja foi
Essa CYRE3 ja foi
Essa ECOR3 ja foi
Essa EGIE3 ja foi
Essa ELET3 ja foi
Essa ELET6 ja foi
Essa EMBR3 ja foi
Essa ENGI11 ja foi
Essa EQTL3 ja foi
Essa EZTC3 ja foi
Essa FLRY3 ja foi
Essa GGBR4 ja foi
Essa GOAU4 ja foi
Essa GOLL4 ja foi
Essa HAPV3 ja foi
Essa HYPE3 ja foi
Essa IRBR3 ja foi
Essa ITSA4 ja foi
Essa ITUB4 ja foi
Essa JBSS3 ja foi
Essa KLBN11 ja foi
Essa LREN3 ja foi
Essa MGLU3 ja foi
Essa MRFG3 ja foi
Essa MRVE3 ja foi
Essa MULT3 ja foi
Essa NTCO3 ja foi
Essa PCAR3 ja foi
Essa PETR3 ja foi
Essa PETR4 ja foi
Essa PRIO3 ja foi
Essa QUAL3 ja foi
Essa RADL3 ja foi
Essa RAIL3 ja foi
Essa RENT3 ja foi
Essa SA

In [20]:
pd.concat(lista_dataframes).tic.unique()

70

In [21]:
pd.concat(lista_dataframes).to_csv('dre_trimestral.csv')

In [48]:
lista_dataframes = []
#setor_financeiro = ['BBAS3','BBDC3']
setor_financeiro = {'BBAS3', 'BBDC3','BBDC4','BBSE3','ITUB4','BPAC11'}
iteracao=0
for acao in codigos_ibovespa:
    if acao in data_frame_feito['tic'].unique():
        lista_dataframes.append(data_frame_feito[data_frame_feito.tic == acao])
        print(f'Essa {acao} ja foi')
    else:
        servico = webdriver.chrome.service.Service(ChromeDriverManager().install())
        # Iniciar o navegador com o serviço
        navegador = webdriver.Chrome(service=servico)
      
        
        if acao in setor_financeiro:
            print(f'Execucao da {acao} do setor financeiro.')
            start_time = time.time() 
            navegador.get(f'https://www.investsite.com.br/principais_indicadores.php?cod_negociacao={acao}')
            #tabela = navegador.find_element(By.XPATH, '//*[@id="tabela_resumo_empresa_dre_3meses"]')

            # Extrair o texto de todas as datas
            elementos_data = navegador.find_elements(By.XPATH, '//*[@id="tabela_resumo_empresa_dre_3meses"]/thead/tr/th[2]/select/option')
            datas = [elemento.text for elemento in elementos_data]

            # Encontrar a tabela com o id
            tabela = navegador.find_element(By.ID, 'tabela_resumo_empresa_dre_3meses')
            receita_liquida = []
            resultado_bruto  = []
            lista_ebit = []
            depreciacao_amortizacao = []
            lista_ebitda = []
            lucro_liquido = []
            lucro_acao = []
            for i in datas:
                elemento_select = navegador.find_element(By.XPATH, '/html/body/div[1]/div[4]/div[2]/div[1]/div[3]/table[2]/thead/tr/th[2]/select')
                # Crie um objeto Select
                select = Select(elemento_select)
                # Altere o período selecionando por texto visível
                select.select_by_visible_text(i)
                servico = webdriver.chrome.service.Service(ChromeDriverManager().install())

                # Pegar as linhas da tabela
                linhas = tabela.find_elements(By.TAG_NAME, 'tr')

                # Iterar pelas linhas e capturar os dados
                lista_dre_ultimo_mes = []
                for linha in linhas:
                    # Pegar as células de cada linha
                    celulas = linha.find_elements(By.TAG_NAME, 'td')
                    for celula in celulas:
                        lista_dre_ultimo_mes.append(celula.text)


                receita_liquida.append(lista_dre_ultimo_mes[1].replace('R$','').replace(',','.').strip())
                resultado_bruto.append(lista_dre_ultimo_mes[3].replace('R$','').replace(',','.').strip())
                lucro_liquido.append(lista_dre_ultimo_mes[7].replace('R$','').replace(',','.').strip())
                lista_ebit.append(np.nan)
                depreciacao_amortizacao.append(np.nan)
                lista_ebitda.append(np.nan)
                lucro_acao.append(np.nan)

        else:
            print(f'Execucao da {acao}')
            start_time = time.time() 
            navegador.get(f'https://www.investsite.com.br/principais_indicadores.php?cod_negociacao={acao}')
            #tabela = navegador.find_element(By.XPATH, '//*[@id="tabela_resumo_empresa_dre_3meses"]')

            # Extrair o texto de todas as datas
            elementos_data = navegador.find_elements(By.XPATH, '//*[@id="tabela_resumo_empresa_dre_3meses"]/thead/tr/th[2]/select/option')
            datas = [elemento.text for elemento in elementos_data]

            # Encontrar a tabela com o id
            tabela = navegador.find_element(By.ID, 'tabela_resumo_empresa_dre_3meses')
            receita_liquida = []
            resultado_bruto  = []
            lista_ebit = []
            depreciacao_amortizacao = []
            lista_ebitda = []
            lucro_liquido = []
            lucro_acao = []
            for i in datas:
                elemento_select = navegador.find_element(By.XPATH, '/html/body/div[1]/div[4]/div[2]/div[1]/div[3]/table[2]/thead/tr/th[2]/select')#/html/body/div[1]/div[4]/div[2]/div[1]/div[3]/table[2]/thead/tr/th[2]/select
                # Crie um objeto Select
                select = Select(elemento_select)
                # Altere o período selecionando por texto visível
                select.select_by_visible_text(i)
                servico = webdriver.chrome.service.Service(ChromeDriverManager().install())

                # Pegar as linhas da tabela
                linhas = tabela.find_elements(By.TAG_NAME, 'tr')

                # Iterar pelas linhas e capturar os dados
                lista_dre_ultimo_mes = []
                for linha in linhas:
                    # Pegar as células de cada linha
                    celulas = linha.find_elements(By.TAG_NAME, 'td')
                    for celula in celulas:
                        lista_dre_ultimo_mes.append(celula.text)
                receita_liquida.append(lista_dre_ultimo_mes[1].replace('R$','').replace(',','.').strip())
                resultado_bruto.append(lista_dre_ultimo_mes[3].replace('R$','').replace(',','.').strip())
                lista_ebit.append(lista_dre_ultimo_mes[5].replace('R$','').replace(',','.').strip())
                depreciacao_amortizacao.append(lista_dre_ultimo_mes[7].replace('R$','').replace(',','.').strip())
                lista_ebitda.append(lista_dre_ultimo_mes[9].replace('R$','').replace(',','.').strip())
                lucro_liquido.append(lista_dre_ultimo_mes[11].replace('R$','').replace(',','.').strip())
                lucro_acao.append(lista_dre_ultimo_mes[13].replace('R$','').replace(',','.').strip())

        listas_dados = {
                'data':datas,
                'receita_liquida': receita_liquida,
                'resultado_bruto': resultado_bruto,
                'ebit': lista_ebit,
                'depreciacao_amortizacao': depreciacao_amortizacao,
                'ebitida': lista_ebitda,
                'lucro_liquido': lucro_liquido,
                'lucro_por_acao': lucro_acao
            }
        df_dre_ultimo_mes = pd.DataFrame(listas_dados)
        df_dre_ultimo_mes['data'] = pd.to_datetime(df_dre_ultimo_mes['data'], format='%d/%m/%Y')
        df_dre_ultimo_mes['data'] = df_dre_ultimo_mes['data'].dt.strftime('%Y-%m-%d')

        if acao in setor_financeiro:
            colunas_selecionadas = df_dre_ultimo_mes.columns[(df_dre_ultimo_mes.columns != 'data') & 
                                        (df_dre_ultimo_mes.columns != 'lucro_por_acao') &
                                        (df_dre_ultimo_mes.columns != 'ebit') &   
                                        (df_dre_ultimo_mes.columns != 'depreciacao_amortizacao') & 
                                        (df_dre_ultimo_mes.columns != 'lucro_por_acao') &
                                        (df_dre_ultimo_mes.columns != 'ebitida') &  
                                        (df_dre_ultimo_mes.columns != 'lucro_por_acao')]
            for col in colunas_selecionadas:
                df_dre_ultimo_mes[col] = df_dre_ultimo_mes[col].apply(converter_valor)   
        else:
            for col in df_dre_ultimo_mes.columns[(df_dre_ultimo_mes.columns != 'data') & (df_dre_ultimo_mes.columns != 'lucro_por_acao')]:
                df_dre_ultimo_mes[col] = df_dre_ultimo_mes[col].apply(converter_valor)

        df_dre_ultimo_mes['tic'] = acao

        lista_dataframes.append(df_dre_ultimo_mes)

        iteracao = iteracao+1

        if iteracao % 15 == 0:
            pd.concat(lista_dataframes).to_csv('dre_trimestral.csv')

        end_time = time.time()

        # Calcula o tempo decorrido em segundos
        execution_time_seconds = end_time - start_time

        # Calcula o tempo decorrido em minutos
        execution_time_minutes = execution_time_seconds / 60

        # Imprime o tempo de execução em minutos
        print(f"Tempo de execução da ação {acao}: {execution_time_minutes:.2f} minutos")               

Essa ABEV3 ja foi
Essa AZUL4 ja foi
Essa B3SA3 ja foi
Essa BBAS3 ja foi
Essa BBDC3 ja foi
Essa BBDC4 ja foi
Essa BBSE3 ja foi
Essa BEEF3 ja foi
Essa BPAC11 ja foi
Essa BRAP4 ja foi
Essa BRFS3 ja foi
Essa BRKM5 ja foi
Essa CCRO3 ja foi
Essa CIEL3 ja foi
Essa CMIG4 ja foi
Essa COGN3 ja foi
Essa CPFE3 ja foi
Essa CRFB3 ja foi
Essa CSAN3 ja foi
Essa CSNA3 ja foi
Essa CVCB3 ja foi
Essa CYRE3 ja foi
Essa ECOR3 ja foi
Essa EGIE3 ja foi
Essa ELET3 ja foi
Essa ELET6 ja foi
Essa EMBR3 ja foi
Essa ENGI11 ja foi
Essa EQTL3 ja foi
Essa EZTC3 ja foi
Execucao da FLRY3
Tempo de execução da ação FLRY3: 2.06 minutos
Execucao da GGBR4
Tempo de execução da ação GGBR4: 3.19 minutos
Execucao da GOAU4
Tempo de execução da ação GOAU4: 3.18 minutos
Execucao da GOLL4
Tempo de execução da ação GOLL4: 2.69 minutos
Execucao da HAPV3
Tempo de execução da ação HAPV3: 0.90 minutos
Execucao da HYPE3
Tempo de execução da ação HYPE3: 2.28 minutos
Execucao da IRBR3


IndexError: list index out of range

In [12]:
servico = webdriver.chrome.service.Service(ChromeDriverManager().install())
# Iniciar o navegador com o serviço
navegador = webdriver.Chrome(service=servico)
#setor_financeiro = ['BBAS3','BBDC3']
setor_financeiro = {'BBAS3', 'BBDC3','BBDC4','ITUB4'}
for acao in ['USIM5']:
    print(f'Execucao da {acao}')
    start_time = time.time() 
    navegador.get(f'https://www.investsite.com.br/principais_indicadores.php?cod_negociacao={acao}')
    #tabela = navegador.find_element(By.XPATH, '//*[@id="tabela_resumo_empresa_dre_3meses"]')

    # Extrair o texto de todas as datas
    elementos_data = navegador.find_elements(By.XPATH, '//*[@id="tabela_resumo_empresa_dre_3meses"]/thead/tr/th[2]/select/option')
    datas = [elemento.text for elemento in elementos_data]

    # Encontrar a tabela com o id
    tabela = navegador.find_element(By.ID, 'tabela_resumo_empresa_dre_3meses')
    receita_liquida = []
    resultado_bruto  = []
    lista_ebit = []
    depreciacao_amortizacao = []
    lista_ebitda = []
    lucro_liquido = []
    lucro_acao = []
    for i in datas:
        elemento_select = navegador.find_element(By.XPATH, '/html/body/div[1]/div[4]/div[2]/div[1]/div[3]/table[2]/thead/tr/th[2]/select')
        # Crie um objeto Select
        select = Select(elemento_select)
        # Altere o período selecionando por texto visível
        select.select_by_visible_text(i)
        servico = webdriver.chrome.service.Service(ChromeDriverManager().install())

        # Pegar as linhas da tabela
        linhas = tabela.find_elements(By.TAG_NAME, 'tr')

        # Iterar pelas linhas e capturar os dados
        lista_dre_ultimo_mes = []
        for linha in linhas:
            # Pegar as células de cada linha
            celulas = linha.find_elements(By.TAG_NAME, 'td')
            for celula in celulas:
                lista_dre_ultimo_mes.append(celula.text)

        if acao in setor_financeiro:
            receita_liquida.append(lista_dre_ultimo_mes[1].replace('R$','').replace(',','.').strip())
            resultado_bruto.append(lista_dre_ultimo_mes[3].replace('R$','').replace(',','.').strip())
            lucro_liquido.append(lista_dre_ultimo_mes[7].replace('R$','').replace(',','.').strip())
            lista_ebit.append(0)
            depreciacao_amortizacao.append(0)
            lista_ebitda.append(0)
            lucro_acao.append(0)
        else:
            receita_liquida.append(lista_dre_ultimo_mes[1].replace('R$','').replace(',','.').strip())
            resultado_bruto.append(lista_dre_ultimo_mes[3].replace('R$','').replace(',','.').strip())
            lista_ebit.append(lista_dre_ultimo_mes[5].replace('R$','').replace(',','.').strip())
            depreciacao_amortizacao.append(lista_dre_ultimo_mes[7].replace('R$','').replace(',','.').strip())
            lista_ebitda.append(lista_dre_ultimo_mes[9].replace('R$','').replace(',','.').strip())
            lucro_liquido.append(lista_dre_ultimo_mes[11].replace('R$','').replace(',','.').strip())
            lucro_acao.append(lista_dre_ultimo_mes[13].replace('R$','').replace(',','.').strip())
            
    listas_dados = {
        'data':datas,
        'receita_liquida': receita_liquida,
        'resultado_bruto': resultado_bruto,
        'ebit': lista_ebit,
        'depreciacao_amortizacao': depreciacao_amortizacao,
        'ebitida': lista_ebitda,
        'lucro_liquido': lucro_liquido,
        'lucro_por_acao': lucro_acao
    }
    df_dre_ultimo_mes_itub = pd.DataFrame(listas_dados)
    df_dre_ultimo_mes_itub['data'] = pd.to_datetime(df_dre_ultimo_mes_itub['data'], format='%d/%m/%Y')
    df_dre_ultimo_mes_itub['data'] = df_dre_ultimo_mes_itub['data'].dt.strftime('%Y-%m-%d')
    
    if acao in setor_financeiro:
        colunas_selecionadas = df_dre_ultimo_mes_itub.columns[(df_dre_ultimo_mes_itub.columns != 'data') & 
                                (df_dre_ultimo_mes_itub.columns != 'lucro_por_acao') &
                                (df_dre_ultimo_mes_itub.columns != 'ebit') &   
                                (df_dre_ultimo_mes_itub.columns != 'depreciacao_amortizacao') & 
                                (df_dre_ultimo_mes_itub.columns != 'lucro_por_acao') &
                                (df_dre_ultimo_mes_itub.columns != 'ebitida') &  
                                (df_dre_ultimo_mes_itub.columns != 'lucro_por_acao')]
        for col in colunas_selecionadas:
            df_dre_ultimo_mes_itub[col] = df_dre_ultimo_mes_itub[col].apply(converter_valor)   
    else:
        for col in df_dre_ultimo_mes_itub.columns[(df_dre_ultimo_mes_itub.columns != 'data') & (df_dre_ultimo_mes_itub.columns != 'lucro_por_acao')]:
            df_dre_ultimo_mes_itub[col] = df_dre_ultimo_mes_itub[col].apply(converter_valor)

    df_dre_ultimo_mes_itub['tic'] = acao
    
    end_time = time.time()
    
    # Calcula o tempo decorrido em segundos
    execution_time_seconds = end_time - start_time

    # Calcula o tempo decorrido em minutos
    execution_time_minutes = execution_time_seconds / 60

    # Imprime o tempo de execução em minutos
    print(f"Tempo de execução da ação {acao}: {execution_time_minutes:.2f} minutos")

Execucao da USIM5
Tempo de execução da ação USIM5: 7.76 minutos


In [14]:
df_dre_ultimo_mes_itub

,data,receita_liquida,resultado_bruto,ebit,depreciacao_amortizacao,ebitida,lucro_liquido,lucro_por_acao,tic
0,2023-09-30,6710.00,175.36,-244.65,-261.51,16.86,-211.98,- 0.17,USIM5
1,2023-06-30,6890.00,582.47,130.46,-262.31,392.77,254.22,0.20,USIM5
2,2023-03-31,7260.00,884.79,540.45,-248.67,789.12,474.13,0.38,USIM5
3,2022-12-31,7660.00,773.15,-1020.00,-234.50,-788.17,-982.15,- 0.78,USIM5
4,2022-09-30,8430.00,1000.00,641.97,-224.95,866.92,517.00,0.41,USIM5
...,...,...,...,...,...,...,...,...,...
89,2001-03-31,679.58,246.41,210.91,NaN,NaN,6.70,0.03,USIM5
90,2000-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NA,USIM5
91,2000-09-30,626.48,248.27,225.41,NaN,NaN,61.81,0.27,USIM5
92,2000-06-30,605.03,224.20,171.38,NaN,NaN,34.00,0.15,USIM5
